## Exploratory Analysis

In [8]:
import pandas as pd
predictive_maintenance = pd.read_csv("predictive_maintenance.csv")
predictive_maintenance.head()

,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9
0,1/1/15,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,1/1/15,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,1/1/15,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,1/1/15,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,1/1/15,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3


In [12]:
predictive_maintenance.describe()

,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9
count,124494.000000,1.244940e+05,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000
mean,0.000851,1.223881e+08,159.484762,9.940455,1.741120,14.222669,260172.657726,0.292528,0.292528,12.451524
std,0.029167,7.045933e+07,2179.657730,185.747321,22.908507,15.943028,99151.078547,7.436924,7.436924,191.425623
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000,8.000000,0.000000,0.000000,0.000000
25%,0.000000,6.128476e+07,0.000000,0.000000,0.000000,8.000000,221452.000000,0.000000,0.000000,0.000000
50%,0.000000,1.227974e+08,0.000000,0.000000,0.000000,10.000000,249799.500000,0.000000,0.000000,0.000000
75%,0.000000,1.833096e+08,0.000000,0.000000,0.000000,12.000000,310266.000000,0.000000,0.000000,0.000000
max,1.000000,2.441405e+08,64968.000000,24929.000000,1666.000000,98.000000,689161.000000,832.000000,832.000000,18701.000000


In [13]:
predictive_maintenance.device.describe()

count       124494
unique        1169
top       Z1F0QK05
freq           304
Name: device, dtype: object

## Model Development

In [2]:
from sklearn.model_selection import train_test_split
feature_cols = ['metric1','metric2','metric3','metric4','metric5','metric6','metric7','metric8','metric9']
X = predictive_maintenance[feature_cols]
y = predictive_maintenance.failure
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.50,random_state=0)

In [4]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
logisticRegr.score(X_test, y_test)

0.9991967484376757

In [6]:
from joblib import dump
dump(logisticRegr, 'model.joblib') 

['model.joblib']

In [15]:
from pydantic import BaseModel
class MaintenanceMetrics(BaseModel):
    metric1: int
    metric2: int
    metric3: int
    metric4: int
    metric5: int
    metric6: int
    metric7: int
    metric8: int
    metric9: int

In [16]:
data = {
 "metric1": 215630672,
  "metric2": 56,
  "metric3": 0,
  "metric4": 52,
  "metric5": 6,
  "metric6": 407438,
  "metric7": 0,
  "metric8": 0,
  "metric9": 7
}

In [26]:
m = MaintenanceMetrics(**data)

,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9
0,215630672,56,0,52,6,407438,0,0,7


In [31]:
logisticRegr.predict(pd.DataFrame.from_dict([m.dict()]))[0]

0

In [36]:
lr_model = load('model.joblib')

In [37]:
lr_model.predict(pd.DataFrame.from_dict([m.dict()]))[0]

0